# Identify gaps in CARE service relative to unsheltered population
* Map number of CARE/CARE+ requests by tract (2017-2019)
* Map number of homeless counts by tract (2017-2019)
* Map difference: (total CARE/CARE+ for homeless encampments) - (total homelessness)

In [17]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import ipyleaflet
from ipyleaflet import Map, GeoData, LayersControl
import json
import requests
from ipywidgets import link, FloatSlider
from branca.colormap import linear

In [18]:
catalog = intake.open_catalog('../catalogs/*.yml')
bucket_name = 's3://city-of-los-angeles-data-lake/public-health-dashboard/'

## Import files

In [19]:
homeless = gpd.read_file(f'{bucket_name}gis/raw/homelessness_lacity_2017_2019.geojson').to_crs({'init':'epsg:4326'})
# Only keep the tracts that are withn City of LA
homeless = homeless[homeless.CD != 0]

In [20]:
pop = pd.read_parquet(f'{bucket_name}data/raw/pop_by_tract2017.parquet') 

In [21]:
care = gpd.read_file(f'{bucket_name}gis/intermediate/care311_tracts.geojson').to_crs({'init':'epsg:4326'})

## Merge dfs

In [22]:
# Merge homelessness and population
m1 = pd.merge(homeless, pop, on = 'GEOID', how = 'inner', validate = 'm:1')

In [23]:
# Merge in CARE 311 service requests
m2 = pd.merge(m1, care, on = ['GEOID', 'year'], how = 'left', validate = '1:1')

In [24]:
# Keep the geometry from homelessness because it's been clipped to City of LA
m2 = m2.drop(columns = ['geometry_y'])

m2.rename(columns = {'geometry_x': 'geometry', 'GEOID': 'id'}, inplace = True)

In [25]:
# Fill in NaNs with zeroes for CARE service requests
for col in ['bulky', 'homeless', 'illegal', 'other']:
    m2[col] = m2[col].fillna(0)    

In [26]:
pivot1 = m2.dissolve(by = ['id', 'clipped_area', 'full_area', 
                           'CD', 'pop', 'SPA', 'SD'], aggfunc = 'sum').reset_index().drop(columns = ['year'])

pivot1.head()

,id,clipped_area,full_area,CD,pop,SPA,SD,geometry,totUnshelt,totShelt,totPeople,bulky,homeless,illegal,other
0,06037101110,0.441019,0.441019,7,4566.0,2,5,"POLYGON ((-118.30229 34.25870, -118.30091 34.2...",29.498,0,29.498,1.0,3.0,0.0,0.0
1,06037101122,1.020722,1.020722,7,3064.0,2,5,"POLYGON ((-118.30334 34.27371, -118.30330 34.2...",4.618,0,4.618,0.0,0.0,0.0,0.0
2,06037101210,0.251197,0.251197,7,6043.0,2,5,"POLYGON ((-118.29945 34.25598, -118.29792 34.2...",41.897,0,41.897,1.0,3.0,6.0,1.0
3,06037101220,0.269802,0.269802,7,3340.0,2,5,"POLYGON ((-118.28592 34.25227, -118.28592 34.2...",72.464,0,72.464,2.0,1.0,1.0,0.0
4,06037101300,0.995075,0.996474,7,4285.0,2,5,"POLYGON ((-118.27822 34.25068, -118.27822 34.2...",36.710,0,36.710,2.0,8.0,5.0,0.0


## Map homelessness

In [27]:
tracts = pivot1[['id', 'geometry']]
tracts['GEOID'] = tracts['id']
tracts = tracts.set_index('id')
tracts.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,geometry,GEOID
id,,
06037101110,"POLYGON ((-118.30229 34.25870, -118.30091 34.2...",06037101110
06037101122,"POLYGON ((-118.30334 34.27371, -118.30330 34.2...",06037101122
06037101210,"POLYGON ((-118.29945 34.25598, -118.29792 34.2...",06037101210
06037101220,"POLYGON ((-118.28592 34.25227, -118.28592 34.2...",06037101220
06037101300,"POLYGON ((-118.27822 34.25068, -118.27822 34.2...",06037101300


In [28]:
# to_json converts the gdf to json. json.loads converts it into dictionary
geo_data = json.loads(tracts.to_json())

# Take what we want to map and turn it into a dictionary
unsheltered = dict(zip(pivot1['id'].tolist(), pivot1['totUnshelt'].tolist()))

layer = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = unsheltered,
    colormap = linear.YlOrRd_04,
    border_color = 'black',
    style = {'fillOpacity': 0.8}
)


m = ipyleaflet.Map(center = (34.0536, -118.2427), zoom = 10)
m.add_layer(layer)

m


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …